In [1]:
import glob
import os
import numpy as np

In [2]:
import json

def parse_file_to_dicts(file_path):
    dicts = []
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('###Iter:'):
                # Extract the JSON string part
                json_str = line.split('::', 1)[1].strip()
                # Use json.loads to convert the string to a dictionary
                dict_data = json.loads(json_str.replace("'", "\""))
                dicts.append(dict_data)
    return dicts

In [9]:
def max_values_from_dicts(dicts):
    max_values = {}
    max_indices = {}

    for index, d in enumerate(dicts):
        for key, value in d.items():
            # Check if we need to update the maximum value and its index for each key
            if key not in max_values or value > max_values[key]:
                max_values[key] = value
                max_indices[key] = index

    # Merge the dictionaries to have the value and index in a tuple
    combined_max_values = {key: (max_values[key], max_indices[key]) for key in max_values}

    return combined_max_values

In [4]:
files = glob.glob("/app/nn-runtime-network/workdir/*/logs.txt")

In [5]:
max_vals={}
for f in files:
    try:
        mv = max_values_from_dicts(parse_file_to_dicts(f))
        if "opa_xla___default" in mv:
            max_vals[f.split("/")[-2]] = mv
    except:
        print("error:",f)

error: /app/nn-runtime-network/workdir/listmle_graphsage_realtivenorm/logs.txt


In [17]:
best_metrics = max_values_from_dicts(max_vals.values())
run_names = list(max_vals.keys())
{k:(v[0],run_names[v[1]]) for k,v in best_metrics.items()}

{'training_loss': (493486359.3808594, 'listmle_mse_graphsage'),
 'ordered_pair_accuracy': (0.7270833253860474,
  'listmle_graphsage_bestparams_layernorm'),
 'kendall_tau': (0.4541666666666666, 'listmle_graphsage_bestparams_layernorm'),
 'valid_loss': (16670388.712890625, 'listmle_mse_graphsage'),
 'opa_nlp___default': (0.7614583373069763,
  'listmle_graphsage_bestparams_layernorm'),
 'ktau_nlp___default': (0.5221152176852893,
  'listmle_graphsage_bestparams_layernorm'),
 'opa_nlp___random': (0.8729166388511658, 'listmle_graphsage_bestparams'),
 'ktau_nlp___random': (0.7458333333333333, 'listmle_graphsage_bestparams'),
 'opa_xla___default': (0.6428571343421936,
  'listmle_graphsage_bestparams_layer5'),
 'ktau_xla___default': (0.2857142857142857,
  'listmle_graphsage_bestparams_layer5'),
 'opa_xla___random': (0.6964285969734192,
  'listmle_graphsage_bestparams_layer5'),
 'ktau_xla___random': (0.3850754560807358,
  'listmle_graphsage_bestparams_layer5')}

In [7]:
np.mean([zz[k] for k in zz if "ktau" in k])

0.4846845732034111